In [1]:
import pandas as pd
from astropy.io import fits
import numpy as np
from pathlib import Path
import glob

from dateutil.parser import parse
import align_moving

from importlib import reload

from os.path import exists

In [3]:
# read in images table

imgs = pd.read_csv('img_master_spreadsheet.csv')

imgs['timestamp'] = imgs['timestamp'].apply(parse)

def _get_aligned_img_path(fp: str):
    return str(Path('./aligned/').joinpath(Path(fp).name.rstrip('.fits') + '_affineremap.fits'))

imgs['aligned_filepath'] = imgs['filepath'].apply(_get_aligned_img_path)
imgs['ok'] = imgs['aligned_filepath'].apply(exists)

okay = imgs['ok'] == True

#imgs.sort_values('timestamp', inplace=True)

imgs
# PLEASE RUN THE FILTER CELL AFTER THIS (see above)

,Unnamed: 0,filepath,timestamp,ref_img,ok,aligned_filepath
0,0,processed/combined_2022-11-13T03:49:33.0000__B...,2022-11-13 03:49:33,./processed/combined_2022-11-13T02:40:26.0000_...,True,aligned/combined_2022-11-13T03:49:33.0000__Bro...
1,2,processed/combined_2022-11-12T07:15:36.0000__B...,2022-11-12 07:15:36,./processed/combined_2022-11-12T06:59:36.0000_...,False,aligned/combined_2022-11-12T07:15:36.0000__Bro...
2,3,processed/combined_2022-11-11T07:16:31.0000__B...,2022-11-11 07:16:31,./processed/combined_2022-11-11T05:51:00.0000_...,False,aligned/combined_2022-11-11T07:16:31.0000__Bro...
3,4,processed/combined_2022-11-12T06:12:20.0000__B...,2022-11-12 06:12:20,./processed/combined_2022-11-12T06:59:36.0000_...,False,aligned/combined_2022-11-12T06:12:20.0000__Bro...
4,5,processed/combined_2022-11-11T06:14:00.0000__B...,2022-11-11 06:14:00,./processed/combined_2022-11-11T05:51:00.0000_...,True,aligned/combined_2022-11-11T06:14:00.0000__Bro...
...,...,...,...,...,...,...
605,656,processed/combined_2022-11-11T06:47:20.0000__B...,2022-11-11 06:47:20,./processed/combined_2022-11-11T05:51:00.0000_...,False,aligned/combined_2022-11-11T06:47:20.0000__Bro...
606,657,processed/combined_2022-11-13T06:35:57.0000__B...,2022-11-13 06:35:57,./processed/combined_2022-11-13T02:40:26.0000_...,True,aligned/combined_2022-11-13T06:35:57.0000__Bro...
607,658,processed/combined_2022-11-13T05:10:40.0000__B...,2022-11-13 05:10:40,./processed/combined_2022-11-13T02:40:26.0000_...,False,aligned/combined_2022-11-13T05:10:40.0000__Bro...
608,659,processed/combined_2022-11-11T07:44:51.0000__B...,2022-11-11 07:44:51,./processed/combined_2022-11-11T05:51:00.0000_...,False,aligned/combined_2022-11-11T07:44:51.0000__Bro...


In [5]:
# FILTER CELL

night_1 = (imgs['timestamp'] >= parse('2022-11-11T05:51:00')) & (imgs['timestamp'] < parse('2022-11-12T00:00:00'))



night_2_group_1 = (
    (parse('2022-11-12T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-12T05:59:20'))
    )

night_2_group_2 = (
    (parse('2022-11-12T05:59:20') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-13T00:00:00'))
    )


night_3 = (
    (parse('2022-11-13T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-14T00:00:00'))
    )


In [19]:
align_moving = reload(align_moving)

# Photometry

In [ ]:
# click on image to print its coordinates and timestamp!
# MAKE SURE TO RUN pip install mplcursors
%matplotlib tk

align_moving.get_coords_of_point(imgs.loc[night_2_group_1 & okay, 'aligned_filepath'].iloc[1])

1444.7405127340194, 584.4306797099007, '2022-11-12T02:42:48.0000'
1445.0230224321135, 583.3006409175242, '2022-11-12T02:42:48.0000'


In [7]:
# NIGHT 1

%matplotlib tk

FWHM = 13

photometry = align_moving.do_photometry_with_timestamps(
    imgs[okay][night_1]['aligned_filepath'],

    start_object_x=723.7147046501888,
    start_object_y=944.1470203183914,
    start_object_t='2022-11-11T06:14:00.0000',

    end_object_x=676.7723412773491,
    end_object_y=948.8662977890997,
    end_object_t='2022-11-11T06:01:00.0000', 

    aperture_=FWHM*3,
    annulus_inner=5*FWHM,
    annulus_outer=9*FWHM,
    dry_run=True,

    reference_positions=[] # CHANGE THIS
 
)

photometry

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_27535/1414686120.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_1]['aligned_filepath'],


IndexError: list index out of range

In [25]:
# Night 2 group 1

%matplotlib tk

FWHM = 10

photometry = align_moving.do_photometry_with_timestamps(
    imgs[okay & night_2_group_1]['aligned_filepath'],

    start_object_x=651,
    start_object_y=983,
    start_object_t='2022-11-12T02:49:48',

    end_object_x=1088,
    end_object_y=938,
    end_object_t='2022-11-12T04:51:31', 

    aperture_=FWHM*3,
    annulus_inner=5*FWHM,
    annulus_outer=9*FWHM,
    dry_run=False,

    reference_positions=[(1009.8564266803644, 511.4699819071131), (1445.0230224321135, 583.3006409175242)] # CHANGE THIS

)

photometry

0      3838.828946
1      4049.702873
2      3955.284480
3      4360.895480
4      4456.608365
          ...     
114    4663.287861
115    4510.844404
116    4246.611487
117    3578.580818
118    3930.402249
Name: aperture_sum_bkgsub, Length: 119, dtype: float64


/Users/student/A101/Project/align_moving.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data["num"].loc[i] = i


,target_subtracted_counts,star_subtracted_counts,num,timestamp,differential,uncalibrated_star_mag,uncalibrated_target_mag,differential_mag


In [8]:
# Night 3

#1211.797141170518, 858.9736043177603, '2022-11-13T06:35:57.0000'

#617.4767807949628, 932.9911452184181, '2022-11-13T03:51:33.0000'

%matplotlib tk

FWHM = 17

photometry = align_moving.do_photometry_with_timestamps(
    imgs[okay][night_3]['aligned_filepath'],

    start_object_x=617.4767807949628,
    start_object_y=932.9911452184181,
    start_object_t='2022-11-13T03:51:33.0000',

    end_object_x=1211.797141170518,
    end_object_y=858.9736043177603,
    end_object_t='2022-11-13T06:35:57.0000', 

    aperture_=FWHM*3,
    annulus_inner=5*FWHM,
    annulus_outer=9*FWHM,
    dry_run=True,

    reference_positions=[] # CHANGE THIS

)

photometry

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_27535/2918774185.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_3]['aligned_filepath'],


IndexError: list index out of range

In [ ]:
imgs[okay][night_2_group_1]['aligned_filepath']

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_23049/2914660617.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_2_group_1]['aligned_filepath']


6      aligned/combined_2022-11-12T04:53:31.0000__Bro...
11     aligned/combined_2022-11-12T04:37:31.0000__Bro...
16     aligned/combined_2022-11-12T02:48:48.0000__Bro...
20     aligned/combined_2022-11-12T02:50:48.0000__Bro...
22     aligned/combined_2022-11-12T04:15:11.0000__Bro...
                             ...                        
595    aligned/combined_2022-11-12T03:19:08.0000__Bro...
598    aligned/combined_2022-11-12T04:23:11.0000__Bro...
599    aligned/combined_2022-11-12T04:18:11.0000__Bro...
602    aligned/combined_2022-11-12T02:45:48.0000__Bro...
603    aligned/combined_2022-11-12T04:00:11.0000__Bro...
Name: aligned_filepath, Length: 119, dtype: object